In [5]:
from astroquery.sdss import SDSS
from astropy import coordinates as coords
from astropy.io import fits
import numpy as np
from PIL import Image
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from astropy.table import Table,vstack,Column,unique
import copy
import os.path
from astropy.visualization import make_lupton_rgb
from reproject import reproject_interp
from __future__ import print_function

In [22]:
# Convert the fits images to jpg images

detected_fields = Table.read('tables/detected_fields_table.data',format='ascii')

num_of_fields = len(detected_fields)

for i in range(2):
    g_band_file_name = 'fits_images/%s.g.fits'%(detected_fields['imageName'][i])
    r_band_file_name = 'fits_images/%s.r.fits'%(detected_fields['imageName'][i])
    i_band_file_name = 'fits_images/%s.i.fits'%(detected_fields['imageName'][i])
    
    jpg_file_name = 'jpg_images/%s.jpg'%(detected_fields['imageName'][i])
    
    g_band_image = fits.open(g_band_file_name)[0]
    r_band_image = fits.open(r_band_file_name)[0]
    i_band_image = fits.open(i_band_file_name)[0]
    
    r_band_image_data = r_band_image.data
    
    # The width and the height of the field image
    image_width = r_band_image_data.shape[1]
    image_height = r_band_image_data.shape[0]
    
    # Reproject the g_band and i_band images to be in the same projection as the r_band image.
    # The footprint is an array that provides information on the footprint of the first image in
    # the new reprojected image plane (essentially which pixels in the new image had a corresponding 
    # pixel in the old image.
    g_band_image_data, g_band_image_footprint = reproject_interp(g_band_image, r_band_image.header)
    i_band_image_data, i_band_image_footprint = reproject_interp(i_band_image, r_band_image.header)
    
    # Return a Red/Green/Blue color image from up to 3 images using an asinh stretch.
    make_lupton_rgb(i_band_image_data, r_band_image_data , g_band_image_data, Q=10, stretch=0.5, filename=jpg_file_name)